In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.io import read_image
import math

test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

test_dir = '../input/petfinder-pawpularity-score/test/'

def create_path(df,root_dir):
    df['Path'] = df['Id'].apply(lambda x: root_dir+x+'.jpg')
    
    
    
    
create_path(test_csv,test_dir)

# Create Label column (used later)
test_csv['Pawpularity'] = 0    

In [3]:
class CustomDataSet(Dataset):
    
    def __init__(self,csv_file,transform=None,augment_transform=None,root_dir ='../input/petfinder-pawpularity-score/train/' ):
        self.csv_file = csv_file
        self.transform = transform
        self.augment_trans = augment_transform
        self.root_dir = root_dir
        self.img_paths = self._get_img_paths(self.csv_file, root_dir)
        
    def __len__(self):
        return int(self.csv_file.shape[0])
    
    def __getitem__(self,idx):
        img_path = self.img_paths[idx]
        image = read_image(img_path)
        label = self.csv_file.Pawpularity.iloc[idx]
        image = image.type(torch.FloatTensor)
        
        if self.transform:
            image = self.transform(image)
        if self.augment_trans:
            image = self.augment_transform(image)
           
        image = torch.mul(image, (1/255))
        return image,label
    
    def _get_img_paths(self,csv_file,root_dir):
        imgs = csv_file['Id'].apply(lambda x: root_dir+x+'.jpg').tolist()
        return imgs

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
                                normalize,
                                transforms.Resize(255),
                                transforms.CenterCrop(224)])
test_data = CustomDataSet(test_csv,transform,root_dir = test_dir)
test_dataloader = DataLoader(test_data,batch_size=64,shuffle=True)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


cuda:0


In [5]:
model_sub1 = torch.load('../input/kfold-pet1/ResNet50_lin_trained_KFOLD1.pth',map_location=torch.device(device)).cuda()
model_sub1 = model_sub1.to(device)

In [6]:
model_sub1.eval()
out_test = torch.tensor([0.00])


for iter2,(inputs,labels) in enumerate(test_dataloader):
    
    model_sub1.eval()
    with torch.no_grad():
        inputs = inputs.to(device)
        inputs = inputs.type(torch.float)
        labels = labels.type(torch.float)
        labels = labels.to(device).view(labels.shape[0],1)


        output1 = model_sub1(inputs).cpu()
        output1 = torch.flatten(output1)
        out_test = torch.cat((out_test,output1))
        out_test = torch.flatten(out_test)
        type(output1)
        
        
        
out_test = out_test[1:]
print(out_test.shape)        

torch.Size([8])


In [7]:
out_test = out_test.numpy().squeeze()


for i in out_test:
    print(i)



47.532135
50.13704
55.94499
51.407402
49.696342
45.660755
51.944515
49.79798


In [8]:


submission["Pawpularity"] = out_test
submission.to_csv("submission.csv", index=False)
submission



,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,47.532135
1,43a2262d7738e3d420d453815151079e,50.137039
2,4e429cead1848a298432a0acad014c9d,55.944988
3,80bc3ccafcc51b66303c2c263aa38486,51.407402
4,8f49844c382931444e68dffbe20228f4,49.696342
5,b03f7041962238a7c9d6537e22f9b017,45.660755
6,c978013571258ed6d4637f6e8cc9d6a3,51.944515
7,e0de453c1bffc20c22b072b34b54e50f,49.797981
